# Fast RCNN for object detection
Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.


**Este Notebook seguirá os seguintes tutoriais:**
- Faster RCNN Train  <br>
https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-train <br>
- Faster RCNN Inference <br>
https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-inference <br>
- Faster RCNN Metric + scrpit details <br>
https://www.kaggle.com/pestipeti/competition-metric-details-script <br>
- TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL <br>
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html <br>
- TRANSFER LEARNING FOR COMPUTER VISION TUTORIAL <br>
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#transfer-learning-for-computer-vision-tutorial

## Declare parâmetros e hiperparâmetros para o modelo
Os componentes podem declarar (e usar) estes parâmetros como padrão:
- dataset
- target

Use estes parâmetros para carregar/salvar conjutos de dados, modelos, métricas e figuras com a ajuda do [SDK da PlatIAgro](https://platiagro.github.io/sdk/). <br>
É possível também declarar parâmetros personalizados para serem definidos ao executar um experimento. 

Selecione os hiperparâmetros e seus respectivos valores para serem usados ao treinar o modelo:
- language

Estes parâmetros são alguns dos oferecidos pela classe do modelo, você também pode utilizar outros existentes. <br>
Dê uma olhada nos [parâmetros do modelo](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer) para mais informações.

In [ ]:
# parâmetros
dataset = "" #@param {type:"string"}
target = "sentiment" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
#Hyperparâametros
train_batch_size = 2 #@param {type:"integer", label:"Tamanho da carga de treino", description:"Tamanho da carga de dados que será utilizada durante o treinamento."}
valid_batch_size = 4 #@param {type:"integer", label:"Tamanho da carga de validação", description:"Tamanho da carga de dados que será utilizada durante o validação."}
test_batch_size = 2 #@param {type:"integer", label:"Tamanho da carga de teste", description:"Tamanho da carga de dados que será utilizada durante o teste."}
max_epochs = 2 #@param {type:"integer", label:"Quantidade máquina de épocas", description:"Quantidade de épocas para o treinamento."}
learning_rate = 0.005 #@param {type:"number", label:"Taxa de aprendizado", description:"Taxa de aprendizado utilizada pelo modelo."}
momentum = 0.9 #@param {type:"number", label:"Momentum", description:"Momentum utilizado pelo otimizador do modelo."}
weight_decay = 0.0005 #@param {type:"number", label:"Taxa de queda do peso", description:"Taxa de queda do peso utilizado durante o treinamento."}
num_classes = 2 #@param {type:"integer", label:"Quantidade de classes", description:"Quantidade de classes."}
detection_threshold = 0.5 #@param {type:"number", label:"Limiar de detecção", description:"Limiar de detecção."}
seed = 7 #@param {type:"integer", label:"Semente", description:"Semente para inicializar geradores de números aleatórios."}

# selected features to perform the model
coord_format = "coco" #@param ["coco","pascal_voc"]  {type:"string", label:"Classes do modelo"}

In [ ]:
###########################################
# ADD FILES PY DOWNLOAD
###########################################

## Extração dos dados do arquivo .zip

In [ ]:
folder = dataset.split('.')[0]

!mkdir -p {folder}
!unzip -o {dataset} -d {folder}

In [ ]:
import os

images = os.listdir(f'{folder}')

images = [x for x in images if not x.startswith('.')]

In [ ]:
## Verifica a existência de GPU
import torch

dev = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(dev)

### Configure quantidade de mensagens

In [ ]:
import logging

logging.getLogger("transformers.configuration_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)
logging.getLogger("lightning").setLevel(logging.WARNING)

### Fixa semente de pesos aleatórios para replicabilidade

In [ ]:
import random

random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

train_df = pd.read_csv(f'{folder}/train.csv')
test_df = pd.read_csv(f'{folder}/sample_submission.csv')

## Manipulação dos dados

In [ ]:
import re
import numpy as np

train_df[['x', 'y', 'w', 'h']] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)
train_df[['x', 'y', 'w', 'h']] = train_df[['x', 'y', 'w', 'h']].astype(np.float)

In [ ]:
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

## Transformações com Albumentations

In [ ]:
from multiprocessing import cpu_count

def get_debug_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
from datasets import WheatDataset

ds_debug = WheatDataset(train_df, DIR_TRAIN, get_debug_transform())
image, target, image_id = ds_debug[0]

## Criação do Dataloader

Collate para colocar batch em tuplas

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

Criando dataloader

In [ ]:
from torch.utils.data import DataLoader, Dataset

debug_loader = DataLoader(
    ds_debug,
    batch_size=4,
    shuffle=False,
    num_workers=cpu_count(),
    collate_fn=collate_fn
)

## Testando um exemplar

In [ ]:
## colocar if para verificar se tem dado de teste

In [ ]:
images, image_ids,targets = next(iter(debug_loader)) 
images = list(image.to(device) for image in images) 
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
boxes = targets[2]['boxes'].cpu().numpy().astype(np.int32)
sample = images[2].permute(1,2,0).cpu().numpy()

In [ ]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

Parâmetros do Modelo

In [ ]:
all_data = [train_df,valid_df,test_df]

hyperparams = {'learning_rate':learning_rate,
               'momentum':momentum,
               'weight_decay':weight_decay,
               'detection_threshold':detection_threshold,
               'train_batch_size':train_batch_size,
               'valid_batch_size':valid_batch_size,
               'test_batch_size':test_batch_size
              }

model_parameters = {'num_classes': num_classes,
                    'coord_format':coord_format
                   }

dataset_infos = {'all_data':all_data,
                 'CustomDataset':WheatDataset,
                 'DIR_TRAIN':DIR_TRAIN,
                 'DIR_TEST':DIR_TEST
                }

extra_infos = {'overfit': False}

Model Finetuner

In [ ]:
from fastrcnn_finetuner import FastRCNNFinetuner

model = FastRCNNFinetuner(hyperparams=hyperparams,
                       model_parameters=model_parameters,
                       dataset_infos=dataset_infos,
                       extra_infos=extra_infos)

sum([torch.tensor(x.size()).prod() for x in model.parameters() if x.requires_grad]) # trainable parameters

In [ ]:
import pytorch_lightning as pl

trainer = pl.Trainer(gpus=0, 
                     checkpoint_callback=False,  # Disable checkpoint saving.
                     fast_dev_run=True)
trainer.fit(model)

In [ ]:
trainer.test(model)

Recuperando ou treinando o modelo

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint


trainer = pl.Trainer(gpus=1,
                     max_epochs=2,
                     check_val_every_n_epoch=1,
                     profiler=True,
                     progress_bar_refresh_rate=1)

model = FastRCNNFinetuner(hyperparams=hyperparams,
                       model_parameters=model_parameters,
                       dataset_infos=dataset_infos,
                       extra_infos = extra_infos) 

trainer.fit(model)

In [ ]:
trainer.test(model)

## Visualização de resultados

In [ ]:
model.df_valid

In [ ]:
model.df_test

## Salva figuras

Utiliza a função `save_figures` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar figuras do [matplotlib](https://matplotlib.org/3.2.1/gallery/index.html). <br>

A avaliação do desempenho do modelo pode ser feita por meio da análise da [Curva ROC (ROC)](https://pt.wikipedia.org/wiki/Caracter%C3%ADstica_de_Opera%C3%A7%C3%A3o_do_Receptor).  Esse gráfico permite avaliar a performance de um classificador binário para diferentes pontos de cortes. A métrica [AUC (Area under curve)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) também é calculada e indicada na legenda do gráfico.<br>
Se a variável resposta tiver mais de duas categorias, o cálculo da curva ROC e AUC é feito utilizando o algoritmo [one-vs-rest](https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics), ou seja, calcula-se a curva ROC e AUC de cada classe em relação ao restante.

In [ ]:
import plots

overlaping_multiple_plots(2, 2, model.result_valid)
inference_multiple_plots(2, 2, model.result_test)

if not weights_retrieved:

    performance_loss_visualization(model.df_performance_train_batch['train_batch_loss'].to_numpy(), epoch_or_batch="Batch",step = "Train")
    performance_loss_visualization(model.df_performance_train_epoch['train_epoch_loss'].to_numpy(), epoch_or_batch="Epoch",step = "Train")
    performance_loss_iou_visualization(model.df_performance_valid_batch['valid_batch_loss'].to_numpy(), model.df_performance_valid_batch['valid_batch_iou'].to_numpy(),epoch_or_batch="Batch",step = "Valid")
    performance_loss_iou_visualization(model.df_performance_valid_epoch['valid_epoch_loss'].to_numpy(), model.df_performance_valid_epoch['valid_epoch_iou'].to_numpy(),epoch_or_batch="Epoch",step = "Valid")
    performance_iou_visualization(model.df_performance_test_batch['test_batch_iou'].to_numpy(), epoch_or_batch="Batch",step = "Test")
    performance_iou_visualization(model.df_performance_test_epoch['test_epoch_iou'].to_numpy(), epoch_or_batch="Epoch",step = "Test")

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [ ]:
file_name = 'pytorch_model.pt'

torch.save(model.state_dict(), f'/tmp/data/{file_name}')

In [ ]:
from joblib import dump

dataset_infos = {'all_data':all_data}

deployment_infos = {'columns':columns,
                    'X_test':X_test
                   }

artifacts = {'hyperparams':hyperparams,
             'model_parameters':model_parameters,
             'dataset_infos':dataset_infos,
             'extra_infos':extra_infos,
             'deployment_infos':deployment_infos
            }

dump(artifacts, "/tmp/data/fasterrcnn.joblib")